## Focal Loss

Hàm mất mát `Focal` là một sự cải tiến so với hàm mất mát `cross-entropy` tiêu chuẩn cho phân loại nhị phân và đa lớp. Nó được giới thiệu trong bài báo có tiêu đề "Focal Loss for Dense Object Detection" của Tsung-Yi Lin và cộng sự, và chủ yếu được thiết kế cho các nhiệm vụ phát hiện đối tượng để giải quyết bài toán `mất cân bằng giữa các lớp` (class imbalance).

Ý chính đằng sau hàm mất mát Focal là giảm trọng số đóng góp của các ví dụ dễ dàng và tập trung vào những ví dụ khó. Điều này giúp ngăn chặn số lượng lớn các ví dụ tiêu cực dễ dàng từ việc áp đặt lên bộ phát hiện trong quá trình đào tạo.

Công thức cho hàm mất mát Focal cho phân loại nhị phân là:

$$ \text{FL}(p_t) = -\alpha_t (1 - p_t)^\gamma \log(p_t) $$

Trong đó:
- $ p_t $ là xác suất của lớp đúng (`true class`). Nếu nhãn lớp đúng là 1, thì $ p_t $ là xác suất dự đoán của mô hình cho lớp 1; nếu nhãn lớp đúng là 0, thì $ p_t = 1 - $ xác suất dự đoán của mô hình cho lớp 1.
- $ \alpha_t $ là một yếu tố cân bằng. Thông thường đặt nằm giữa 0 và 1. Số này được sử dụng để xử lý mất cân bằng lớp.
- $ \gamma $ là tham số tập trung (`focusing parameter`) mục đích điều chỉnh mức độ tập trung vào lớp dễ dàng phân loại. Khi $ \gamma = 0 $, hàm mất mát Focal tương đương với hàm mất mát cross-entropy. Khi $ \gamma $ tăng, hiệu ứng của yếu tố điều chỉnh trở nên rõ ràng hơn.

Lợi thế chính của hàm mất mát Focal là nó đưa ra nhiều trọng số hơn cho các ví dụ bị phân loại sai và ít trọng số hơn cho các ví dụ được phân loại tốt. Điều này giúp trong các tình huống mà một số lớp bị đại diện ít hơn hoặc khi mô hình có khả năng bị áp đặt bởi các ví dụ tiêu cực dễ dàng.

### Tham số $ \alpha $ trong hàm Focal-loss


1. **Mục đích**: $ \alpha $ được sử dụng trong hàm mất mát Focal để xử lý sự mất cân bằng lớp bằng cách điều chỉnh mất mát cho các lớp dương và âm một cách khác nhau. Nó cung cấp một sự cân bằng giữa tầm quan trọng của lớp dương và lớp âm trong việc tính toán mất mát.

2. **Ảnh hưởng lên Giá trị Mất mát**:
   - Đối với lớp dương (tức là khi nhãn thực $ y = 1 $): Giá trị mất mát được nhân với hệ số $ \alpha $.
   - Đối với lớp âm (tức là khi nhãn thực $ y = 0 $): Giá trị mất mát được nhân với hệ số $ 1 - \alpha $.

3. **Phạm vi Giá trị**:
   - Thông thường, $ \alpha $ nằm trong khoảng [0, 1].
   - Giá trị $ \alpha $ càng gần 1 càng làm cho mất mát lớp dương được tăng cường và mất mát cho lớp âm được giảm đi.
   - Ngược lại, một giá trị $ \alpha $ gần 0 sẽ nhấn mạnh hơn đến lớp âm.

4. **Lợi ích**:
   - Bằng cách điều chỉnh $ \alpha $, người ta có thể cung cấp trọng số nhiều hơn cho các lớp được đại diện ít hơn. Điều này có thể đặc biệt hữu ích trong các tình huống có sự mất cân bằng lớp nghiêm trọng, như trong các nhiệm vụ phát hiện đối tượng khi số lượng các ví dụ âm  vượt trội so với các ví dụ dương.
   - Nó đảm bảo rằng mô hình không thiên vị về lớp có nhiều quan sát hơn và xem xét cả hai lớp khi cập nhật trọng số trong quá trình huấn luyện.

5. **Thiết lập $ \alpha $**:
   - Trong một số trường hợp, $ \alpha $ có thể được thiết lập dựa trên phân phối lớp nghịch đảo. Ví dụ, nếu 80% các ví dụ là âm và 20% là dương, người ta có thể đặt $ \alpha $ thành 0,2 cho lớp âm và 0,8 cho lớp dương.
   - Trong những trường hợp khác, giá trị của $ \alpha $ có thể được xác định thông qua kiểm định chéo (cross-validation) hoặc các phương pháp điều chỉnh tham số khác.

Tóm lại, tham số $ \alpha $ trong hàm mất mát Focal cung cấp một cơ chế để xử lý sự mất cân bằng lớp bằng cách điều chỉnh mất mát cho các ví dụ dương và âm một cách khác nhau. Nó đảm bảo rằng cả hai lớp chính và phụ đều được đại diện đầy đủ trong quá trình đào tạo của mô hình.

### Tham số $ \gamma $ trong hàm Focal-loss

$ \gamma $ được gọi là "tham số tập trung". Nó đóng một vai trò quan trọng trong việc xác định mức độ mà mô hình nên tập trung vào các lớp bị phân loại sai so với những lớp được phân loại đúng.

Tác động của nó:

1. **Mục đích**: Mục đích chính của tham số $ \gamma $ trong hàm mất mát Focal là giảm ảnh hưởng của các lớp dễ phân loại và tăng tầm quan trọng của việc hiệu chỉnh các lớp bị phân loại sai. Điều này đặc biệt hữu ích trong các tình huống mà tập dữ liệu có sự mất cân bằng giữa các lớp.

2. **Tác động của việc Thay đổi $ \gamma $**: Thuật ngữ $ (1 - p_t)^\gamma $ trong hàm mất mát Focal là yếu tố điều chỉnh. Ở đây, $ p_t $ đại diện cho xác suất dự đoán của lớp đúng.
   - Nếu $ p_t $ gần bằng 1, nghĩa là ví dụ dễ dàng được phân loại, và $ (1 - p_t)^\gamma $ sẽ gần bằng 0, đặc biệt khi $ \gamma > 0 $.
   - Nếu $ p_t $ xa 1 (tức là dự đoán là không chính xác hoặc không chắc chắn), thì $ (1 - p_t)^\gamma $ sẽ lớn hơn, tăng ảnh hưởng của ví dụ đó lên hàm mất mát.
   - $ \gamma = 0 $: Hàm mất mát Focal giảm xuống còn bằng hàm mất mát cross-entropy tiêu chuẩn, vì yếu tố điều chỉnh trở thành 1 cho tất cả các ví dụ.
   - $ \gamma > 0 $: Tăng trọng số của các lớp khó phân loại và giảm trọng số của những lớp dễ phân loại. $ \gamma $ càng lớn, mô hình càng tập trung nhiều vào các lớp khó.

3. **Lợi ích**: Bằng cách điều chỉnh $ \gamma $, hàm mất mát Focal cho phép các mô hình, đặc biệt trong các nhiệm vụ phát hiện đối tượng, trở nên mạnh mẽ hơn trước số lượng lớn các lớp dễ. Thay vì tiêu tốn tài nguyên tính toán cho các lớp dễ dàng, mô hình tập trung nhiều hơn vào các lớp khó, thường chứa nhiều thông tin hơn.

Tóm lại, tham số $ \gamma $ trong hàm mất mát Focal cung cấp một cơ chế để nhấn mạnh việc học từ các lớp bị phân loại sai so với những lớp dễ phân loại. Đó là một công cụ để xử lý sự mất cân bằng lớp và đảm bảo rằng mô hình chú ý nhiều hơn đến các lớp mà nó phân loại sai.

## Ví dụ

In [11]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import optuna
import lightgbm as lgb
from sklearn.metrics import accuracy_score

# Load the breast cancer dataset from sklearn (binary classification)
data = datasets.load_breast_cancer()
X = data.data
y = data.target
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Define Focal Loss for LightGBM
# def focal_loss_lgb(y_pred, dtrain, alpha=0.25, gamma=2.0):
#     a,g = alpha, gamma
#     y_true = dtrain.get_label()
#     p = 1/(1+np.exp(-y_pred))
#     loss = -( a*y_true + (1-a)*(1-y_true) ) * (( 1 - ( y_true*p + (1-y_true)*(1-p)) )**g) * ( y_true*np.log(p)+(1-y_true)*np.log(1-p) )
#     grad = p * (y_true - 1) + y_true
#     hess = p * (1-p)
#     return grad, hess

# Optuna Study
def objective(trial):
    
    # LightGBM hyperparameters
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_float ('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float ('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }
    
    # Focal Loss hyperparameters
    alpha = trial.suggest_float ('alpha', 0.01, 1)
    gamma = trial.suggest_float ('gamma', 0.1, 5)
    
    def focal_loss_lgb_eval_error(y_pred, dtrain, alpha=alpha, gamma=gamma):
        a,g = alpha, gamma
        y_true = dtrain.get_label()
        p = 1/(1+np.exp(-y_pred))
        loss = -( a*y_true + (1-a)*(1-y_true) ) * (( 1 - ( y_true*p + (1-y_true)*(1-p)) )**g) * ( y_true*np.log(p)+(1-y_true)*np.log(1-p) )
        return 'focal_loss', np.mean(loss), False

    train_set = lgb.Dataset(X_train, y_train)
    val_set = lgb.Dataset(X_val, y_val, reference=train_set)
    
    model = lgb.train(param, train_set, valid_sets=[val_set], callbacks=[lgb.log_evaluation(2)], feval=focal_loss_lgb_eval_error)
    preds = model.predict(X_val, num_iteration=model.best_iteration)
    pred_labels = np.rint(preds)
    accuracy = accuracy_score(y_val, pred_labels)
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

results_optuna = {
    'Number of finished trials': len(study.trials),
    'Best trial value': study.best_trial.value,
    'Best trial params': study.best_trial.params
}

# results_optuna

[I 2023-07-28 17:34:49,580] A new study created in memory with name: no-name-fc716844-ca6c-4e10-a029-eb8062c4f249
[I 2023-07-28 17:34:49,617] Trial 0 finished with value: 0.9649122807017544 and parameters: {'lambda_l1': 3.3418514275590367, 'lambda_l2': 1.5027218058168852, 'num_leaves': 82, 'feature_fraction': 0.6017421406885948, 'bagging_fraction': 0.513781493351129, 'bagging_freq': 7, 'min_child_samples': 86, 'alpha': 0.06866799414757477, 'gamma': 3.146149738108615}. Best is trial 0 with value: 0.9649122807017544.
[I 2023-07-28 17:34:49,657] Trial 1 finished with value: 0.9649122807017544 and parameters: {'lambda_l1': 4.962718089772645, 'lambda_l2': 5.74630711096055, 'num_leaves': 255, 'feature_fraction': 0.8804203477262255, 'bagging_fraction': 0.42228685198590676, 'bagging_freq': 1, 'min_child_samples': 69, 'alpha': 0.8238339251870125, 'gamma': 3.245518942939893}. Best is trial 0 with value: 0.9649122807017544.
[I 2023-07-28 17:34:49,694] Trial 2 finished with value: 0.97368421052631

[2]	valid_0's binary_logloss: 0.556328	valid_0's focal_loss: 0.0854965
[4]	valid_0's binary_logloss: 0.477191	valid_0's focal_loss: 0.0768443
[6]	valid_0's binary_logloss: 0.419844	valid_0's focal_loss: 0.0703484
[8]	valid_0's binary_logloss: 0.367666	valid_0's focal_loss: 0.0645085
[10]	valid_0's binary_logloss: 0.328572	valid_0's focal_loss: 0.0600815
[12]	valid_0's binary_logloss: 0.301224	valid_0's focal_loss: 0.0568992
[14]	valid_0's binary_logloss: 0.274757	valid_0's focal_loss: 0.0539797
[16]	valid_0's binary_logloss: 0.25231	valid_0's focal_loss: 0.0522293
[18]	valid_0's binary_logloss: 0.237393	valid_0's focal_loss: 0.0509419
[20]	valid_0's binary_logloss: 0.222643	valid_0's focal_loss: 0.0498397
[22]	valid_0's binary_logloss: 0.209508	valid_0's focal_loss: 0.0485011
[24]	valid_0's binary_logloss: 0.199834	valid_0's focal_loss: 0.0471112
[26]	valid_0's binary_logloss: 0.188935	valid_0's focal_loss: 0.0459167
[28]	valid_0's binary_logloss: 0.182575	valid_0's focal_loss: 0.04509

[I 2023-07-28 17:34:49,814] Trial 5 finished with value: 0.9649122807017544 and parameters: {'lambda_l1': 4.308894330147343, 'lambda_l2': 7.612636971420471, 'num_leaves': 36, 'feature_fraction': 0.892944131200536, 'bagging_fraction': 0.5633338766409516, 'bagging_freq': 2, 'min_child_samples': 11, 'alpha': 0.8615309698863295, 'gamma': 3.109970592184184}. Best is trial 2 with value: 0.9736842105263158.
[I 2023-07-28 17:34:49,856] Trial 6 finished with value: 0.956140350877193 and parameters: {'lambda_l1': 6.862723440308688, 'lambda_l2': 7.906618548903391, 'num_leaves': 127, 'feature_fraction': 0.9843827542708213, 'bagging_fraction': 0.8935045465903533, 'bagging_freq': 6, 'min_child_samples': 28, 'alpha': 0.3396204633001546, 'gamma': 4.458412266184998}. Best is trial 2 with value: 0.9736842105263158.
[I 2023-07-28 17:34:49,892] Trial 7 finished with value: 0.9649122807017544 and parameters: {'lambda_l1': 7.025334327888933, 'lambda_l2': 0.7347888615502999, 'num_leaves': 7, 'feature_fractio

[2]	valid_0's binary_logloss: 0.562495	valid_0's focal_loss: 0.0206781
[4]	valid_0's binary_logloss: 0.486601	valid_0's focal_loss: 0.0187397
[6]	valid_0's binary_logloss: 0.435064	valid_0's focal_loss: 0.0174048
[8]	valid_0's binary_logloss: 0.387971	valid_0's focal_loss: 0.0161998
[10]	valid_0's binary_logloss: 0.350017	valid_0's focal_loss: 0.0152461
[12]	valid_0's binary_logloss: 0.314349	valid_0's focal_loss: 0.0143589
[14]	valid_0's binary_logloss: 0.286507	valid_0's focal_loss: 0.0136739
[16]	valid_0's binary_logloss: 0.264687	valid_0's focal_loss: 0.0131462
[18]	valid_0's binary_logloss: 0.244388	valid_0's focal_loss: 0.0126582
[20]	valid_0's binary_logloss: 0.224548	valid_0's focal_loss: 0.0121845
[22]	valid_0's binary_logloss: 0.209308	valid_0's focal_loss: 0.011819
[24]	valid_0's binary_logloss: 0.196312	valid_0's focal_loss: 0.0115142
[26]	valid_0's binary_logloss: 0.18999	valid_0's focal_loss: 0.0113554
[28]	valid_0's binary_logloss: 0.18119	valid_0's focal_loss: 0.0111456